In [ ]:
from ursina import *

class Test_cube(Entity):
    def __init__(self):
        super().__init__(
            model='cube',
            color=color.white,
            texture='white_cube',
            rotation= Vec3(45,45,45)
        )
class Test_button(Button):
    def __init__(self):
        super().__init__(
            parent=scene,
            model='cube',
            texture='brick',
            color=color.blue,
            highlight_color=color.red,
            pressed_color=color.lime
        )
    def input(self,key):
        if self.hovered:
            print('boron precionado')
def update():
    if held_keys['a']:
        test_square.x -= 4 * time.dt
    

app = Ursina()

test_square=Entity(model = 'quad',color=color.red,scale=(1,4),position=(5,4))

#sans_texture= load_texture('fondo3.JPEG')
sans=Entity(model='quad',texture= 'assets/Sans.png')

#text_cube=Test_cube()
text_cube=Test_button()
app.run()

In [ ]:
from ursina import *
from ursina.prefabs.first_person_controller import FirstPersonController #simulador de una persona

app= Ursina()

grass_texture= load_texture('![imagenes/grass_block.png](attachment:imagenes/grass_block.png)')
#stone_texture= load_texture('/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/stone_block.png')
#brick_texture= load_texture('/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/brick_block.png')
#dirt_texture= load_texture('/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/dirt_block.png')
class Voxel(Button):
    def __init__(self,position=(0,0,0)):
        super().__init__(
            parent=scene,
            position=position,
            model='cube',
            origin_y=0.5,
            texture=grass_texture, #'white_cube',
            color=color.white,
            highlight_color=color.lime
            
        )
    def input(self,key):  #elevaciones de cajones
        if self.hovered:
            if key == 'left mouse down':
                voxel=Voxel(position=self.position+mouse.normal)
            if key == 'right mouse down':
                destroy(self)
                

for z in range(20):
    for x in range(20):
        voxel=Voxel(position=(x,0,z))
player=FirstPersonController()
app.run()

ursina version: 3.4.0
package_folder: /home/RobertoPacho/anaconda3/lib/python3.8/site-packages/ursina
asset_folder: /home/RobertoPacho/anaconda3/lib/python3.8/site-packages
blender_paths:
{}
OS: posix
screen resolution: (1920, 1080)
size; LVector2f(1536, 864)
render mode: default
no settings.py file
info: psd-tools3 not installed
no settings.py file
development mode: True
application successfully started
no filter quad
changed aspect ratio: 1.777 -> 1.778


#/home/RobertoPacho/Documentos/IA/Tareas_IA_SE/Proyecto/imagenes/grass_block.png
